## dataLoad

In [119]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import pandas as pd
import os
import random
import warnings
warnings.filterwarnings('ignore')
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import numpy as np
from utils import lgbm_predict, post_slack
base_feats = ['KnowledgeTag', 'user_correct_answer', 'user_total_answer', 
         'user_acc', 'test_mean', 'test_sum', 'tag_mean','tag_sum']
# train과 test 데이터셋은 사용자 별로 묶어서 분리를 해주어야함

## 1. 데이터 로딩
data_dir = '/opt/ml/input/data' # 경로
# train_file_path = os.path.join(data_dir, 'train_data.csv') # 데이터
# test_file_path = os.path.join(data_dir, 'test_data.csv')
after_fe_path = os.path.join(data_dir, 'after_fe_train_test.pkl')
df = pd.read_pickle(after_fe_path)

train_df = df[df.kind=='train']
train, valid = custom_train_test_split(train_df, ratio=0.7, seed=2) # 훈련데이터 split
test = df[df.kind=='test'] # 테스트 데이터
train2 = test[test.answerCode!=-1] # 테스트데이터 마지막 제출 2번쨰꺼까지 훈련데이터로 사용
train = pd.concat([train_df,train2]) # 훈련데이터 병합
train.shape, valid.shape, test.shape

x_train = train.drop('answerCode',axis=1)
y_train = train[['answerCode']]

x_valid = valid.drop('answerCode',axis=1)
y_valid = valid[['answerCode']]
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

### 시드별 Validation set align

In [121]:
for seed in range(100):
    print(f"seed: {seed}")
    train, valid = custom_train_test_split(train_df, ratio=0.7, seed=seed) # 훈련데이터 split
    test = df[df.kind=='test'] # 테스트 데이터
    train2 = test[test.answerCode!=-1] # 테스트데이터 마지막 제출 2번쨰꺼까지 훈련데이터로 사용
    train = pd.concat([train_df,train2]) # 훈련데이터 병합
    train.shape, valid.shape, test.shape

    x_train = train.drop('answerCode',axis=1)
    y_train = train[['answerCode']]

    x_valid = valid.drop('answerCode',axis=1)
    y_valid = valid[['answerCode']]
    x_train.shape, y_train.shape, x_valid.shape, y_valid.shape
    FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec'
        ]
    preds = model4.predict(x_valid[FEATS])
    acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
    auc = roc_auc_score(y_valid, preds)

    print(f'3600 VALID AUC : {auc} ACC : {acc}\n')
    # 사용할 Feature 설정
    FEATS = ['KnowledgeTag',
             'user_correct_answer',
             'user_total_answer',
             'big_category',
             'mid_category',
             'problem_num',
             'month','day','dayname','hour',
             'user_acc',
             'test_mean',
             'test_sum',
             'test_std',
             'tag_std',
             'tag_mean',
             'tag_sum',
             'solvesec',
             'time_category'
            ]
    preds = model5.predict(x_valid[FEATS])
    acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
    auc = roc_auc_score(y_valid, preds)

    print(f'4800 VALID AUC : {auc} ACC : {acc}\n')

seed: 0
[(5361, 21), (2201, 537), (5811, 72), (5089, 70), (1372, 789), (2217, 828), (6646, 41), (3390, 276), (1986, 390), (1250, 575), (334, 1162), (1609, 554), (211, 721), (188, 445), (7019, 21), (4098, 144), (1277, 491), (3898, 271), (4311, 144), (2759, 613)]


((2525956, 25), (1968, 25), (260114, 25))

((2525956, 24), (2525956, 1), (1968, 24), (1968, 1))

3600 VALID AUC : 0.8309100018567869 ACC : 0.7479674796747967

4800 VALID AUC : 0.8295978033901399 ACC : 0.7469512195121951

seed: 1
[(345, 591), (4395, 301), (7272, 19), (1284, 1155), (3930, 319), (3585, 344), (6746, 39), (1686, 262), (6166, 40), (431, 978), (6586, 61), (6872, 50), (4807, 216), (148, 666), (5025, 305), (4201, 307), (517, 1119), (5168, 191), (5043, 69), (6801, 52)]


((2525956, 25), (2015, 25), (260114, 25))

...

3600 VALID AUC : 0.8197994272745637 ACC : 0.739433117851815

4800 VALID AUC : 0.8202113459839906 ACC : 0.739433117851815

seed: 3
[(4329, 192), (6443, 24), (5557, 128), (3757, 250), (6749, 25), (4030, 333), (4816, 54), (5284, 146), (1357, 430), (3457, 232), (5971, 38), (1988, 409), (3039, 230), (2398, 187), (5108, 156), (1867, 256), (2579, 385), (3074, 124), (5189, 222), (436, 1304)]


((2525956, 25), (2009, 25), (260114, 25))

((2525956, 24), (2525956, 1), (2009, 24), (2009, 1))

3600 VALID AUC : 0.8372251140774101 ACC : 0.7590841214534594

4800 VALID AUC : 0.835474803858215 ACC : 0.7605774016923843

seed: 4
[(2089, 433), (2421, 480), (2496, 148), (4641, 134), (4046, 155), (4145, 243), (6030, 59), (4769, 133), (6389, 48), (541, 550), (1948, 553), (4933, 157), (7132, 26), (6240, 56), (2914, 550), (230, 800), (2090, 402), (6922, 23), (6661, 51), (4357, 112)]


KeyboardInterrupt: 

In [4]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec'
        ]

params = {
#     "max_depth": 8,  # 8,
#     "min_data_in_leaf": 1000,
    # "feature_fraction": 0.6,  # 0.8,
#     "bagging_fraction": 0.75,
    # "max_cat_group": 64,
    "objective": "binary",
#     "boosting": "gbdt",  # dart
#     "learning_rate": 0.01,  # 0.01,
    # "bagging_freq": 5,
    "seed": 42,
    # "max_bin": 50,
#     "num_leaves": 80,  # 40,
#     "metric": "auc",
}

model_auc = lgb.LGBMClassifier(
    **params,
    n_estimators=3200,
    silent=-1,
)


## 문제푼 시간 초로만 추가한 것

In [5]:
lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

model2 = lgb.train(
    params, 
    lgb_x_train,
    valid_sets=[lgb_x_valid],
    verbose_eval=100,
    num_boost_round=3200,
#     early_stopping_rounds=200,
)

preds = model2.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

post_slack("done")

[LightGBM] [Info] Number of positive: 1653588, number of negative: 872368
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.238892 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3095
[LightGBM] [Info] Number of data points in the train set: 2525956, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639491
[LightGBM] [Info] Start training from score 0.639491
[100]	valid_0's binary_logloss: 0.582419
[200]	valid_0's binary_logloss: 0.575168
[300]	valid_0's binary_logloss: 0.568893
[400]	valid_0's binary_logloss: 0.563229
[500]	valid_0's binary_logloss: 0.559829
[600]	valid_0's binary_logloss: 0.556637
[700]	valid_0's binary_logloss: 0.553144
[800]	valid_0's binary_logloss: 0.550667
[900]	valid_0's binary_logloss: 0.548314
[1000]	valid_0's binary_logloss: 0.546135
[1100]	valid_0's binary_logloss: 0.542614
[1200]	valid_0's binary_logloss: 0.540631
[1300

## 문제푼 시간 4800초 이상 모두 4800초로 변환

In [8]:
lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

model3 = lgb.train(
    params, 
    lgb_x_train,
    valid_sets=[lgb_x_valid],
    verbose_eval=100,
    num_boost_round=3200,
#     early_stopping_rounds=200,
)

preds = model3.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

[LightGBM] [Info] Number of positive: 1653588, number of negative: 872368
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.067944 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3095
[LightGBM] [Info] Number of data points in the train set: 2525956, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639491
[LightGBM] [Info] Start training from score 0.639491
[100]	valid_0's binary_logloss: 0.56096
[200]	valid_0's binary_logloss: 0.554025
[300]	valid_0's binary_logloss: 0.547694
[400]	valid_0's binary_logloss: 0.542811
[500]	valid_0's binary_logloss: 0.536767
[600]	valid_0's binary_logloss: 0.533314
[700]	valid_0's binary_logloss: 0.530218
[800]	valid_0's binary_logloss: 0.527721
[900]	valid_0's binary_logloss: 0.525034
[1000]	valid_0's binary_logloss: 0.522923
[1100]	valid_0's binary_lo

In [48]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec'
        ]
preds = model3.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

VALID AUC : 0.8302605814985996 ACC : 0.7531645569620253



In [9]:
post_slack("done")

In [19]:
lgbm_predict(test, model3, FEATS, 'LGBM_Baseline_문제푼시간_추가_4800초로수렴.csv')

writing prediction : output/LGBM_Baseline_문제푼시간_추가_4800초로수렴.csv


## 문제푼 시간 3600초 이상 모두 3600으로 변환

In [11]:
lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

model4 = lgb.train(
    params, 
    lgb_x_train,
    valid_sets=[lgb_x_valid],
    verbose_eval=100,
    num_boost_round=3200,
#     early_stopping_rounds=200,
)

preds = model4.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

post_slack("done")

lgbm_predict(test, model4, FEATS, 'LGBM_Baseline_문제푼시간_추가_3600초이상모두3600으로.csv')

[LightGBM] [Info] Number of positive: 1653588, number of negative: 872368
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.052752 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3095
[LightGBM] [Info] Number of data points in the train set: 2525956, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639491
[LightGBM] [Info] Start training from score 0.639491
[100]	valid_0's binary_logloss: 0.57308
[200]	valid_0's binary_logloss: 0.564389
[300]	valid_0's binary_logloss: 0.559807
[400]	valid_0's binary_logloss: 0.554239
[500]	valid_0's binary_logloss: 0.550381
[600]	valid_0's binary_logloss: 0.547333
[700]	valid_0's binary_logloss: 0.542663
[800]	valid_0's binary_logloss: 0.539448
[900]	valid_0's binary_logloss: 0.537293
[1000]	valid_0's binary_logloss: 0.533369
[1100]	valid_0's binary_lo

### validation

In [115]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec'
        ]
preds = model4.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

VALID AUC : 0.8194181098835474 ACC : 0.7370959961408586



In [49]:
lgbm_predict(test, model4, FEATS, 'LGBM_Baseline_문제푼시간_추가_3600초이상모두3600으로.csv')

writing prediction : output/LGBM_Baseline_문제푼시간_추가_3600초이상모두3600으로.csv


## 문제푼 시간 카테고리 변수도 추가

In [16]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec',
         'time_category'
        ]

In [17]:
lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

model5 = lgb.train(
    params, 
    lgb_x_train,
    valid_sets=[lgb_x_valid],
    verbose_eval=100,
    num_boost_round=3200,
#     early_stopping_rounds=200,
)

preds = model5.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

[LightGBM] [Info] Number of positive: 1653588, number of negative: 872368
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.052055 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3105
[LightGBM] [Info] Number of data points in the train set: 2525956, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639491
[LightGBM] [Info] Start training from score 0.639491
[100]	valid_0's binary_logloss: 0.560899
[200]	valid_0's binary_logloss: 0.552517
[300]	valid_0's binary_logloss: 0.547221
[400]	valid_0's binary_logloss: 0.540192
[500]	valid_0's binary_logloss: 0.535174
[600]	valid_0's binary_logloss: 0.531541
[700]	valid_0's binary_logloss: 0.527031
[800]	valid_0's binary_logloss: 0.523995
[900]	valid_0's binary_logloss: 0.522249
[1000]	valid_0's binary_logloss: 0.519503
[1100]	valid_0's binary_l

### validation

In [116]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec',
         'time_category'
        ]
preds = model5.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

VALID AUC : 0.818123880262765 ACC : 0.7370959961408586



In [18]:
post_slack("done")

In [52]:
lgbm_predict(test, model5, FEATS, 'LGBM_Baseline_문제푼시간_추가_최대3600으로_time_category추가.csv')

writing prediction : output/LGBM_Baseline_문제푼시간_추가_최대3600으로_time_category추가.csv
